In [33]:
import numpy as np
import os
import pandas as pd
import geopandas as gpd
from IPython.display import display, Markdown
from tqdm import tqdm
import time

In [34]:
rtpSuffix       = 'RTP2023'

dirWork                    = os.getcwd()
dirInput                   = os.path.join(dirWork ,        r'input'                  )
dirATOdata                 = os.path.join(dirInput,        r'TDM_ATO_Output_RTP_v900')
dirIntermediate            = os.path.join(dirWork,         r'intermediate'           )
dirResults                 = os.path.join(dirWork,         r'results'                )
dirIntermediateTravelSheds = os.path.join(dirIntermediate, r'travelsheds'            )

print("Working Directory: "      + dirWork          )
print("Input Directory: "        + dirInput         )
print("Intermediate Directory: " + dirIntermediate  )
print("Results Directory: "      + dirResults       )

gdb_Process_name         = 'process_travelsheds.gdb'
gdb_Process              = os.path.join(dirIntermediate, 'process_travelsheds.gdb')
gdb_ATO_TravelSheds_name = 'TravelSheds_' + rtpSuffix + '.gdb'
gdb_ATO_TravelSheds      = os.path.join(dirResults, r'TravelSheds_' + rtpSuffix + '.gdb')

#name of TAZs
filenameTAZshp               = os.path.join(dirResults, 'TAZ_ATO_' + rtpSuffix + '.shp')
filenameUtahshp              = os.path.join(dirIntermediate, 'UtahSimple.shp')

listTDMs       = ['v9_SE23_Net23','v9_RTP_SE50_Net50']
listLayerNames = ['2023','2050']
listMode       = ['AUTO','TRAN']

Working Directory: e:\GitHub\ATO-Web-App\_dataprep
Input Directory: e:\GitHub\ATO-Web-App\_dataprep\input
Intermediate Directory: e:\GitHub\ATO-Web-App\_dataprep\intermediate
Results Directory: e:\GitHub\ATO-Web-App\_dataprep\results


In [35]:
skip = False

if not skip:

    df_TS_Summary = pd.DataFrame()

    i = -1

    #loop through tdm results from preprocessed data
    for tdm in listTDMs:
        
        i = i + 1 

        filenameTravelShed_Summary = os.path.join(dirIntermediateTravelSheds, tdm + '_TAZOpportunitySheds_Summary.csv')
        display(filenameTravelShed_Summary)

        df_TS_Summary_read = pd.read_csv(filenameTravelShed_Summary, index_col=False)
        
        df_TS_Summary_read['TDM'] = listLayerNames[i]
    
        df_TS_Summary = pd.concat([df_TS_Summary,df_TS_Summary_read])

    display(df_TS_Summary)
    df_TS_Summary.to_json(os.path.join(dirResults, 'TAZSheds_Summary_' + rtpSuffix + '.json'),orient='records',double_precision=2)


'e:\\GitHub\\ATO-Web-App\\_dataprep\\intermediate\\travelsheds\\v9_SE23_Net23_TAZOpportunitySheds_Summary.csv'

'e:\\GitHub\\ATO-Web-App\\_dataprep\\intermediate\\travelsheds\\v9_RTP_SE50_Net50_TAZOpportunitySheds_Summary.csv'

,Z,HHs10Min_Auto,HHs20Min_Auto,HHs30Min_Auto,EMP10Min_Auto,EMP20Min_Auto,EMP30Min_Auto,HHs10Min_Tran,HHs20Min_Tran,HHs30Min_Tran,EMP10Min_Tran,EMP20Min_Tran,EMP30Min_Tran,TDM
0,1,429,8750,6685,172,15281,9212,0,0,0,0,0,0,2023
1,2,612,8970,7173,347,15239,10312,0,0,0,0,0,0,2023
2,3,114,10093,27553,1341,14367,43548,0,0,0,0,0,0,2023
3,4,107,9737,20866,840,14823,35454,0,0,0,0,0,0,2023
4,5,2077,7835,10420,876,14879,12659,0,0,0,0,0,0,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,3625,0,0,0,0,0,0,0,0,1988,0,0,1691,2050
3625,3626,39,42,66,0,28,8,0,0,74,0,0,110,2050
3626,3627,0,0,0,0,0,0,0,0,1096,0,0,538,2050
3627,3628,12347,60142,62732,19321,47516,85695,0,457,5270,0,1192,8366,2050


In [37]:
# Load TAZ shapefile as GeoDataFrame
taz_gdf = gpd.read_file(filenameTAZshp).to_crs(epsg=4326)

# Load Utah shapefile as GeoDataFrame
utah_gdf = gpd.read_file(filenameUtahshp).to_crs(epsg=4326)

output_dir = 'results/travelsheds'

for tdm, layer_name in zip(listTDMs, listLayerNames):
    print(f"Processing TDM: {tdm}")
    for mode in listMode:
        filename_travel_shed = os.path.join(dirIntermediateTravelSheds, f"{tdm}_TAZOpportunitySheds_{mode}.csv")
        print(f"Processing file: {filename_travel_shed}")

        # Read CSV as DataFrame
        df = pd.read_csv(filename_travel_shed)

        # Pre-group and filter TAZs
        grouped_dfs = {taz_id: group for taz_id, group in df.groupby('I')}
        relevant_taz_gdf = taz_gdf[taz_gdf['TAZID'].isin(grouped_dfs.keys())]

        output_gdfs = []  # To accumulate results

        # Initialize progress bar
        with tqdm(total=len(grouped_dfs), desc=f"Processing mode {mode}") as pbar:
            for taz_id, taz_df in grouped_dfs.items():
                # Merge and check if merge produces results
                i_taz_gdf = pd.merge(relevant_taz_gdf, taz_df, left_on='TAZID', right_on='J')
                if i_taz_gdf.empty:
                    # add mask
                    mask_gdf = gpd.GeoDataFrame(
                        {'I': [taz_id], 'SHED': [0]},  # Set attributes explicitly
                        geometry=[utah_gdf.unary_union],  # Wrap in a list
                        crs=utah_gdf.crs               # Maintain the CRS
                    )
                    # Append result
                    output_gdfs.append(mask_gdf)
                    pbar.update(1)
                else:

                    # Dissolve geometries
                    dissolved_gdf = i_taz_gdf.dissolve(by=['I', 'SHED'], as_index=False)
                    dissolved_gdf['I'] = taz_id
                    dissolved_gdf['SHED'] = dissolved_gdf['SHED'].astype(int)

                    # Filter valid geometries
                    valid_gdf = dissolved_gdf[~dissolved_gdf.geometry.isnull() & ~dissolved_gdf.geometry.is_empty]

                    if not valid_gdf.empty:

                        # Append dissolved GeoDataFrame to the list
                        output_gdfs.append(dissolved_gdf)

                        # Add mask which is cutout form entire state
                        dissolved_geom = dissolved_gdf.unary_union
                        utah_geom = utah_gdf.unary_union
                        sym_diff_geom = utah_geom.symmetric_difference(dissolved_geom)

                        # Create GeoDataFrame for symmetric difference with attributes
                        sym_diff_gdf = gpd.GeoDataFrame(
                            {'I': [taz_id], 'SHED': [0]},  # Set attributes explicitly
                            geometry=[sym_diff_geom],      # Assign symmetric difference geometry
                            crs=utah_gdf.crs               # Maintain the CRS
                        )

                        # Append result
                        output_gdfs.append(sym_diff_gdf)

                # Update progress bar
                pbar.update(1)

        # Write accumulated GeoDataFrames to GeoJSON after each mode
        if output_gdfs:
            final_gdf = gpd.GeoDataFrame(pd.concat(output_gdfs, ignore_index=True))
            final_gdf = final_gdf[['I','SHED','geometry']]
            final_gdf = final_gdf.to_crs(epsg=4326)
            output_path = os.path.join(output_dir, f"TravelShed_{layer_name}_{mode}.geojson")
            print ('Output file ' + output_path)
            final_gdf.to_file(output_path, driver="GeoJSON")


Processing TDM: v9_SE23_Net23
Processing file: e:\GitHub\ATO-Web-App\_dataprep\intermediate\travelsheds\v9_SE23_Net23_TAZOpportunitySheds_AUTO.csv


Processing mode AUTO: 3693it [03:09, 19.53it/s]                           


Output file results/travelsheds\TravelShed_2023_AUTO.geojson
Processing file: e:\GitHub\ATO-Web-App\_dataprep\intermediate\travelsheds\v9_SE23_Net23_TAZOpportunitySheds_TRAN.csv


Processing mode TRAN: 2834it [00:42, 67.26it/s]                           


Output file results/travelsheds\TravelShed_2023_TRAN.geojson
Processing TDM: v9_RTP_SE50_Net50
Processing file: e:\GitHub\ATO-Web-App\_dataprep\intermediate\travelsheds\v9_RTP_SE50_Net50_TAZOpportunitySheds_AUTO.csv


Processing mode AUTO: 3692it [03:04, 20.06it/s]                           


Output file results/travelsheds\TravelShed_2050_AUTO.geojson
Processing file: e:\GitHub\ATO-Web-App\_dataprep\intermediate\travelsheds\v9_RTP_SE50_Net50_TAZOpportunitySheds_TRAN.csv


Processing mode TRAN: 3179it [00:50, 62.76it/s]                          


Output file results/travelsheds\TravelShed_2050_TRAN.geojson
